# Setting up connection and cursor

In [1]:
import sqlite3
conn = sqlite3.connect("enquete.db")
c = conn.cursor()

In [90]:
import random
from faker import Faker
from datetime import datetime, timedelta
fake = Faker('fr_FR')

In [15]:
conn.close()

# Running querries

In [118]:
c.execute("DROP TABLE IF EXISTS employés")
c.execute("DROP TABLE IF EXISTS vacances_2023")
c.execute("DROP TABLE IF EXISTS badges")
c.execute("DROP TABLE IF EXISTS mails")
c.execute("DROP TABLE IF EXISTS congés_2023")

In [120]:
c.execute("""
CREATE TABLE employés (
id INTEGER PRIMARY KEY AUTOINCREMENT, 
nom VARCHAR, 
prénom VARCHAR,
fonction VARCHAR,
salaire INTEGER,
date_embauche VARCHAR
)""")
c.execute("""
CREATE TABLE congés_2023 (
    date_départ_demandée VARCHAR,
    date_départ_accordée VARCHAR,
    date_retour_demandée VARCHAR,
    date_retour_accordée VARCHAR,
    id_employé INTEGER,
    FOREIGN KEY (id_employé) REFERENCES employés(id)
)""")
c.execute("""
CREATE TABLE badges (
    date_heure_badge VARCHAR,
    lieu_badge VARCHAR,
    type_badge VARCHAR,
    id_employé INTEGER,
    FOREIGN KEY (id_employé) REFERENCES employés(id)
)""")
c.execute(
"""CREATE TABLE mails (
    date TEXT,
    contenu TEXT,
    id_expéditeur INTEGER,
    id_destinataire INTEGER,
    FOREIGN KEY (id_expéditeur) REFERENCES employés(id),
    FOREIGN KEY (id_destinataire) REFERENCES employés(id)
)""")
conn.commit()

In [123]:
time = fake.date_time_between(start_date="8",end_date="21")

ParseError: Can't parse date string `8`

In [115]:
def génération_congés():
    data = []
    ids_employés = list(range(1001))
    for i in range(1001):
        date_départ_demandée = fake.date_between(start_date="-1y",end_date="today")
        proba = random.randint(0,10)
        if proba in [0,1,2,3,4,5,6,7,8]:
            date_départ_accordée = date_départ_demandée
        else:
            date_départ_accordée = timedelta(days=random.randint(-10,10))
            date_départ_accordée = date_départ_demandée + date_départ_accordée
        temps_congé = random.choice((7,14))
        temps_congé = timedelta(days=temps_congé)
        date_retour_demandée = date_départ_demandée + temps_congé
        date_retour_accordée = date_départ_accordée + temps_congé
        id_employé = random.choice(ids_employés)
        ids_employés.remove(id_employé)
        data.append([date_départ_demandée,date_départ_accordée,date_retour_demandée,date_retour_accordée,id_employé])
    return data



In [114]:
ids = list(range(1,10))
for i in range(1,10):
    id = random.choice(ids)
    print(id)
    ids.remove(id)

1
4
3
9
5
6
7
8
2


In [105]:
a = list(range(1,10))
a.remove(1)

In [121]:
insert_query = """INSERT INTO congés_2023 (date_départ_demandée, date_départ_accordée, date_retour_demandée, 
                date_retour_accordée, id_employé) VALUES (?,?,?,?,?)"""
c.executemany (insert_query, génération_congés())
conn.commit()

### PAS POSSIBLE DE FAIRE COMME CI-DESSOUS ÇA MALHEUREUSEMENT 

In [27]:
employés = """
CREATE TABLE employés (
id INTEGER PRIMARY KEY AUTOINCREMENT, 
nom VARCHAR, 
prénom VARCHAR,
fonction VARCHAR,
salaire INTEGER,
date_embauche VARCHAR
);"""
vacances_2023 = """
CREATE TABLE vacances_2023 (
    date_départ VARCHAR,
    date_retour VARCHAR,
    date_pose VARCHAR,
    id_employé INTEGER,
    FOREIGN KEY (id_employé) REFERENCES employés(id)
);"""
badges = """
CREATE TABLE vacances_2023 (
    date_départ VARCHAR,
    date_retour VARCHAR,
    date_pose VARCHAR,
    id_employé INTEGER,
    FOREIGN KEY (id_employé) REFERENCES employés(id)
);"""
mails = """
CREATE TABLE mails (
    date 
    contenu TEXT
    id_expéditeur INTEGER
    id_destinataire INTEGER
    FOREIGN KEY (id_expéditeur) REFERENCES employés(id)
    FOREIGN KEY (id_destinataire) REFERENCES employés(id)
)
"""
c.execute(employés+vacances_2023+badges)

ProgrammingError: You can only execute one statement at a time.

In [98]:
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker('fr_FR')  # Utiliser le français comme langue

def generate_employee(index):
    first_name = fake.first_name()
    last_name = fake.last_name()
    
    # Assurer qu'il n'y a qu'un seul PDG
    if index == 0:
        function = 'PDG'
    else:
        function = random.choice(['Ingénieur TAL', 'Informaticien réseau', 'Expert machine learning','Développeur','Chef de projet'])
    
    salary = round(random.uniform(2000, 150000))  # Salaire aléatoire entre 50000 et 120000
    hire_date = fake.date_between(start_date='-5y', end_date='today')
    return [last_name, first_name, function, salary, hire_date]

employees = [generate_employee(i) for i in range(1000)]

# Afficher les 10 premiers employés pour vérifier
for employee in employees[:10]:
    print(employee)



['Lévêque', 'Denise', 'PDG', 49863, datetime.date(2019, 5, 18)]
['Benard', 'Margaud', 'Expert machine learning', 75907, datetime.date(2022, 3, 3)]
['Bazin', 'Marguerite', 'Expert machine learning', 132800, datetime.date(2020, 11, 9)]
['Tessier', 'Benoît', 'Ingénieur TAL', 100876, datetime.date(2019, 1, 13)]
['Bonnet', 'Gérard', 'Informaticien réseau', 45087, datetime.date(2020, 1, 5)]
['Salmon', 'Zoé', 'Chef de projet', 72774, datetime.date(2021, 6, 7)]
['Bouvier', 'Julie', 'Ingénieur TAL', 7169, datetime.date(2022, 6, 11)]
['Diallo', 'Victor', 'Développeur', 42198, datetime.date(2020, 5, 3)]
['Pruvost', 'Lorraine', 'Chef de projet', 125792, datetime.date(2019, 1, 30)]
['Leclercq', 'Mathilde', 'Expert machine learning', 26257, datetime.date(2018, 12, 30)]


In [99]:
insert_query = "INSERT INTO employés(nom, prénom, fonction, salaire, date_embauche) VALUES (?,?,?,?,?)"

c.executemany (insert_query, employees)
conn.commit()

In [101]:
insert_query = "INSERT INTO mails (date, contenu, id_expéditeur, id_destinataire) VALUES (?,?,?,?)"
c.executemany (insert_query, mails)
conn.commit()

In [ ]:
print(len(employees))

In [ ]:
date = str(fake.date_between(start_date="-1y", end_date='today'))
time = fake.time()
date += f"|{time}"
print(date)

In [122]:
def génération_mails():
    mails = []
    ouverture = ["Cher collègue,\n\n","Salut\n\n","Bonjour\n\n"]
    fermeture = ["Bien à toi\n\n","Bien cordialement\n\n","À +","À tout à l'heure"]
    for i in range(0,40000):
        date = str(fake.date_between(start_date="-1y", end_date='today'))
        time = fake.time()
        date += f"|{time}"
        id_expéditeur = random.randint(1,1000)
        id_destinataire = random.randint(1,1000)
        texte = random.choice(ouverture)
        texte += fake.text()
        texte += random.choice(fermeture)
        mails.append([date,texte,id_expéditeur,id_destinataire])
    return mails

In [5]:
date_départ_demandée = fake.date_between(start_date="-1", end_date="today")

In [2]:
query_vacances = """
SELECT id
FROM employés
INNER JOIN congés_2023 ON employés.id = congés_2023.id_employé
WHERE "2023-04-01" BETWEEN date_départ_accordée AND date_retour_accordée
"""
res = c.execute(query_vacances)
disculpés = [elem[0]for elem in res.fetchall()]
suspects = list(range(5001))
suspects = [suspect for suspect in suspects if suspect not in disculpés]
suspects = ", ".join(map(str,suspects))
query_prénoms = f"SELECT prénom FROM employés WHERE id IN ({suspects})"
prénoms = c.execute(query_prénoms)
prénoms = [elem[0] for elem in prénoms.fetchall()]



NameError: name 'c' is not defined